# Notebook dedicated to getting water mask using isce and mintpy
- https://github.com/insarlab/MintPy-tutorial/blob/main/applications/water_mask.ipynb

In [ ]:
import os
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from typing import List, Union
from itertools import combinations
import eof
import geemap
import csv
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, box
import glob
import subprocess